## Loading packages
Required packages for downstream analysis

In [1]:
inst <- suppressMessages(lapply(c('DESeq2',
                                  'vidger',
                                  'pheatmap',
                                  'ggplot2',
                                  'EnhancedVolcano',
                                  'VennDiagram',
                                  'org.Mm.eg.db',
                                  'clusterProfiler',
                                  'UpSetR',
                                  'readxl',
                                  'tidyverse',
                                  'KEGGREST'),
                                library,
                                character.only=TRUE)
) 

Set whether to produce plots, set to False for test runs.

In [2]:
bool_plot = FALSE

In [3]:
prot_input_file = '~/Documents/consultation/Luiza/omics_data/data/astrocyteprotoeme12102017-1.xlsx'
options(repr.plot.width=8, repr.plot.height=8)
my_palette <- colorRampPalette(c("blue", "white", "red"))(n=255)

In [4]:
dat <- read.csv(file="/Users/viktorian.miok/Documents/data/RNA-seq_Data-NEW/new_counts1.csv", 
                header=TRUE,
                sep=",")
rownames(dat) <- dat[,1]
dat <- dat[,-1]

In [5]:
group <- c(rep("HPT_Chow",6), rep("HPT_HFD",7), rep("Hippo_Chow",6), 
           rep("Hippo_HFD",7), rep("CTX_Chow",6), rep("CTX_HFD",7))
lab <- colnames(dat)
loc <- c(rep("Hypothalamus",13), rep("Hippocampus",13), rep("Cortex",13))
diet <- c(rep("chow",6), rep("hfd",7), rep("chow",6), rep("hfd",7), rep("chow",6), rep("hfd",7))
id <- colnames(dat)
metaData <- data.frame(id, group, loc,diet)

In [6]:
# remove probleatic samples
dat <- dat[,-c(10,13,19,34)] #dat[,-c(10,13,19,34,36)]    19,,36  #10,13,19,32,39
metaData <- metaData[-c(10,13,19,34),] #metaData[-c(10,13,19,34,36),]    19,36

In [7]:
# convert from ensembl to symbol gene id
rownames(dat)<-gsub("\\..*","",rownames(dat))
sig.gene<-bitr(gsub("\\..*","",rownames(dat)), fromType="ENSEMBL", toType="SYMBOL", OrgDb=org.Mm.eg.db)

symb <- sig.gene$SYMBOL[ match(rownames(dat), sig.gene$ENSEMBL) ]
dat <- dat[!is.na(symb),]
symb <- symb[!is.na(symb)]
rownames(dat) <- make.names(symb,TRUE)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(gsub("\\..*", "", rownames(dat)), fromType = "ENSEMBL", :
“50.55% of input gene IDs are fail to map...”


In [8]:
# filtering
keep <- rowSums(dat) > 10
dat <- dat[keep,]

keep <- rowSums(dat >= 10) >= 5
dat <- dat[keep,]

## Comparison of diet effect for each tissue

In [9]:
dds<-DESeqDataSetFromMatrix(countData=dat, colData = metaData, design=~ group)
dds<-DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 155 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [10]:
# variance stabilizing transformation data
vsd <- assay(vst(dds, 
                 blind=TRUE
             )
)

In [11]:
options(repr.plot.width=10, repr.plot.height=8)

In [12]:
if(bool_plot){
    pcaData <- plotPCA(vst(dds[,1:11]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 <- dds@colData$group[1:11]
    percentVar <- round(100 * attr(pcaData, "percentVar"))

    p <- ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
                geom_point(aes(color=gr1), size=5) + 
                geom_point(shape=1, color="black", size=5) + 
                xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
                ylab(paste0("PC2: ",percentVar[2],"% variance")) +
                theme(axis.text=element_text(size=20), 
                      axis.title=element_text(size=20), 
                      legend.title=element_text(siz=20),
                      legend.text=element_text(size=20),
                      plot.title=element_text(size=20)) + 
                scale_color_manual(values=c("chocolate","tan1")) +
                ylim(-25,15) + xlim(-20,20)
    p
}

In [13]:
if(bool_plot){
    pcaData <- plotPCA(vst(dds[,12:23]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 <- metaData$group[12:23]
    percentVar <- round(100 * attr(pcaData, "percentVar"))

    p <- ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
                geom_point(aes(color=gr1), size=5) + 
                geom_point(shape = 1, color = "black", size = 5) + 
                xlab(paste0("PC1: ",percentVar[1], "% variance")) + 
                ylab(paste0("PC2: ",percentVar[2], "% variance")) +
                theme(axis.text = element_text(size=20), 
                      axis.title = element_text(size=20), 
                      legend.title = element_text(siz=20),
                      legend.text = element_text(size=20),
                      plot.title = element_text(size=20)) + 
                scale_color_manual(values = c("mediumseagreen", "lightgreen"))
    p 
}

In [14]:
if(bool_plot){
    pcaData <- plotPCA(vst(dds[,24:35]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 <- metaData$group[24:35]
    percentVar <- round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
           geom_point(aes(color=gr1), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",percentVar[1], "% variance")) + 
           ylab(paste0("PC2: ",percentVar[2], "% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("lightblue4", "lightblue"))
}

In [15]:
if(bool_plot){ 
    df_pca <- prcomp(t(vsd))

    df_out <- as.data.frame(df_pca$x)
    df_out$tissue <- as.factor(metaData$group) 

    ggplot(df_out, aes(x=PC1, y=PC2, color=tissue)) + 
           geom_point(aes(color=tissue), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",round(df_pca$sdev[1]), "% variance")) + 
           ylab(paste0("PC2: ",round(df_pca$sdev[2]), "% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("lightblue", "lightblue4", "tan1",
                                       "chocolate", "lightgreen", "mediumseagreen"))
}

In [16]:
options(repr.plot.width=10, repr.plot.height=10)

In [18]:
pdf(file="PCA.pdf", width=10, height=10)
    pcaData <- plotPCA(vst(dds),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    percentVar <- round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=metaData$group)) +
           geom_point(aes(color=metaData$group), size=10) + 
           geom_point(shape=1, color="black", size=10) + 
           xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
           ylab(paste0("PC2: ",percentVar[2],"% variance")) +
           theme(axis.text=element_text(size=25), 
                 axis.title=element_text(size=30)) +
           scale_color_manual(values=c("aquamarine4", "aquamarine2", "darkorange2",
                                       "orange", "blueviolet", "mediumpurple1")) +
           theme(legend.position="none")
dev.off()

pdf 
  2

In [19]:
# Heatmap of all samples and genes
if(bool_plot){  
    agsh <- pheatmap(vsd,
                     cluster_rows=T, 
                     scale="row", 
                     show_rownames=FALSE, 
                     color=my_palette,
                     cluster_cols=F, 
                     annotation_col=as.data.frame(colData(dds)[,c("diet", "loc")]),
                     fontsize=10,
                     breaks=seq(-3, 3, length.out=255)
    )
}

In [20]:
hpt<-results(dds, contrast=c("group", "HPT_HFD", "HPT_Chow"))
hippo<-results(dds, contrast=c("group", "Hippo_HFD", "Hippo_Chow"))
ctx<-results(dds, contrast=c("group", "CTX_HFD", "CTX_Chow"))

hpt<-hpt[complete.cases(hpt), ]
hippo<-hippo[complete.cases(hippo), ]
ctx<-ctx[complete.cases(ctx), ]

### HPT

In [21]:
nrow(hpt[(hpt$pvalue<0.05)&(hpt$log2FoldChange<0),])
nrow(hpt[(hpt$pvalue<0.05)&(hpt$log2FoldChange>0),])
nrow(hpt[(hpt$pvalue<0.05),])

[1] 569

[1] 581

[1] 1150

### CTX

In [22]:
nrow(ctx[(ctx$pvalue<0.05)&(ctx$log2FoldChange<0),])
nrow(ctx[(ctx$pvalue<0.05)&(ctx$log2FoldChange>0),])
nrow(ctx[(ctx$pvalue<0.05),])

[1] 1510

[1] 1309

[1] 2819

### Hippo

In [23]:
nrow(hippo[(hippo$pvalue<0.05)&(hippo$log2FoldChange<0),])
nrow(hippo[(hippo$pvalue<0.05)&(hippo$log2FoldChange>0),])
nrow(hippo[(hippo$pvalue<0.05),])

[1] 488

[1] 270

[1] 758

In [24]:
pdf(file="volcano_hpt.pdf", width=10, height=10)
    EnhancedVolcano(hpt,
                    lab=rownames(hpt),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [25]:
pdf(file="volcano_hippo.pdf", width=10, height=10)
    EnhancedVolcano(hippo,
                    lab=rownames(hippo),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [26]:
pdf(file = "volcano_ctx.pdf", width = 10, height = 10)
    EnhancedVolcano(ctx,
                    lab = rownames(ctx),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [27]:
if(bool_plot){
    df <- data.frame(Tissue=c("CTX", "HPT", "Hippo"),
                     Nr_significant_genes=c(sum(ctx[,5]<0.05,na.rm=T),
                                            sum(hpt[,5]<0.05,na.rm=T), 
                                            sum(hippo[,5]<0.05,na.rm=T))
    )
    ggplot(df, aes(x=Tissue, y=Nr_significant_genes, fill=Tissue))  +
           geom_bar(stat="identity") + 
            theme(axis.text=element_text(size=20), 
                  axis.title=element_text(size=20), 
                  legend.title=element_text(siz=20),
                  legend.text=element_text(size=20),
                  plot.title=element_text(size=20)) +
           theme(legend.position="none") +
           scale_fill_manual(values=c("lightblue", "tan1", "lightgreen"))
}

In [28]:
if(bool_plot){
    df <- data.frame(Tissue=c("CTX", "HPT", "Hippo"),
                     Nr_significant_genes=c(sum(ctx[,5]<0.01, na.rm=T), 
                                            sum(hpt[,5]<0.01, na.rm=T),
                                            sum(hippo[,5]<0.01, na.rm=T))
    )
    ggplot(df, aes(x=Tissue, y=Nr_significant_genes, fill=Tissue)) +
           geom_bar(stat="identity") + 
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) +
           theme(legend.position="none") +
           scale_fill_manual(values=c("lightblue", "tan1", "lightgreen"))
}


In [29]:
options(repr.plot.width=8, repr.plot.height=8)

In [30]:
if(bool_plot){
    pdf(file="venn_tran.pdf", width=8, height=8)
    vp<-venn.diagram(list(' '=rownames(hpt[hpt[,5]<0.05,]), 
                          ' '=rownames(hippo[hippo[,5]<0.05,]),
                          ' '=rownames(ctx[ctx[,5]<0.05,])),
                     fill=c("orange", "mediumpurple1", "aquamarine2"),
                     filename=NULL,
                     cex=3.5, 
                     col="transparent",
                     cat.cex=0.01,
                     margin=0.01,
                     main.cex=3,
                     main=NULL);
    grid.draw(vp)  
    dev.off()
}

# Comparison between tissues

In [31]:
dds<-DESeqDataSetFromMatrix(countData=dat,
                            colData=metaData,
                            design=~ loc
)
dds<-DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 325 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [32]:
hpt_vs_hippo<-results(dds, contrast=c("loc", "Hypothalamus", "Hippocampus"))
hpt_vs_ctx<-results(dds, contrast=c("loc", "Hypothalamus", "Cortex"))
ctx_vs_hippo<-results(dds, contrast=c("loc", "Cortex", "Hippocampus"))

In [33]:
if(bool_plot){
    EnhancedVolcano(hpt_vs_hippo,
                    lab=rownames(hpt_vs_hippo),
                    pCutoff=0.05,
                    FCcutoff=15,
                    x='log2FoldChange',
                    y='pvalue',
                    ylim=c(0, 100),
                    xlim=c(-12, 12),
                    col=c("grey30", "grey30", "red2", "red2"),
                    title="HPT vs. Hippo"
    )
}

In [34]:
if(bool_plot){
    EnhancedVolcano(hpt_vs_ctx,
                    lab=rownames(hpt_vs_ctx),
                    pCutoff=0.05,
                    FCcutoff=15,
                    x='log2FoldChange',
                    y='pvalue',
                    ylim=c(0, 100),
                    xlim=c(-12, 12),
                    col=c("grey30", "grey30", "red2", "red2"),
                    title="HPT vs. CTX"
    )
}

In [35]:
if(bool_plot){
    EnhancedVolcano(ctx_vs_hippo,
                    lab=rownames(ctx_vs_hippo),
                    pCutoff=0.05,
                    FCcutoff=15,
                    x='log2FoldChange',
                    y='pvalue',
                    ylim=c(0, 100),
                    xlim=c(-12, 12),
                    col=c("grey30", "grey30", "red2", "red2"),
                    title="CTX vs. Hippo"
    )
}

In [36]:
hpt_vs_hippo1=hpt_vs_hippo[which(hpt_vs_hippo[,5]<0.05),]
hpt_vs_ctx1=hpt_vs_ctx[which(hpt_vs_ctx[,5]<0.05),]
ctx_vs_hippo1=ctx_vs_hippo[which(ctx_vs_hippo[,5]<0.05),]

In [37]:
if(bool_plot){
    df <- data.frame(location=c("HPT_vs_Hippo", "HPT_vs_CTX", "CTX_vs_Hippo"),
                     Sig_genes=c(sum(hpt_vs_hippo[,5]<0.05,na.rm=T), 
                                 sum(hpt_vs_ctx[,5]<0.05,na.rm=T), 
                                 sum(ctx_vs_hippo[,5]<0.05,na.rm=T))
    )
    ggplot(df, aes(x=location, y=Sig_genes, fill=location))  +
           geom_bar(stat="identity") +
           theme_minimal() +
           theme(legend.position="none") +
           scale_fill_manual(values=c("purple", "brown", "blue"))
}

# Integrative Analysis Transcriptomics and Proteomics Data

In [38]:
# load the results from Persus
cnam<-c("GeneName", "pvalue", "log2FoldChange")
diffHippo<-read_excel(prot_input_file, sheet=10)[,c(4, 6, 7)]
colnames(diffHippo)<-cnam
for (i in 1:nrow(diffHippo)) diffHippo[i,2]<-10 ^ -diffHippo[i,2]

# load the results from Persus
diffCTX <- read_excel(prot_input_file, sheet=8)[,c(4,6,7)]
colnames(diffCTX)<-cnam
for (i in 1:nrow(diffCTX)) diffCTX[i,2]<-10 ^ -diffCTX[i,2]

# load the results from Persus
diffHPT <- read_excel(prot_input_file, sheet=6)[,c(4,6,7)]
colnames(diffHPT)<-cnam
for (i in 1:nrow(diffHPT)) diffHPT[i,2]<-10 ^ -diffHPT[i,2]

In [39]:
# cut-off
cutoff=0.05

In [40]:
diffCTX1<-diffCTX[which(diffCTX$pvalue<cutoff),]
diffHippo1<-diffHippo[which(diffHippo$pvalue<cutoff),]
diffHPT1<-diffHPT[which(diffHPT$pvalue<cutoff),]

In [41]:
cnam1<-c("GeneName", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj")
hippo[,1]=rownames(hippo)
colnames(hippo)=cnam1
hippo<-as.data.frame(hippo)

hpt[,1]=rownames(hpt)
colnames(hpt)=cnam1
hpt<-as.data.frame(hpt)

ctx[,1]=rownames(ctx)
colnames(ctx)=cnam1
ctx<-as.data.frame(ctx)

In [42]:
ctx1<-ctx[which(ctx$pvalue<cutoff),]
hippo1<-hippo[which(hippo$pvalue<cutoff),]
hpt1<-hpt[which(hpt$pvalue<cutoff),]

In [43]:
if(bool_plot){
    listInput <- list(CTX_transciptomics=rownames(ctx1), 
                      Hippo_transciptomics=rownames(hippo1),
                      HPT_transciptomics=rownames(hpt1),
                      CTX_proteomics=as.matrix(diffCTX1[,1]),
                      Hippo_proteomics=as.matrix(diffHippo1[,1]),
                      HPT_proteomics=as.matrix(diffHPT1[,1])
    )
    upset(fromList(listInput), 
          order.by="freq", 
          text.scale=c(2.5, 2.5, 2.5, 2.5, 2, 2.5),
          nsets=10
    )
}

In [44]:
if(bool_plot){
#pdf(file = "venn_ctx.pdf", width=8, height=8)
vp <- venn.diagram(list(t=rownames(ctx1),
                        p=as.matrix(diffCTX1[,1])),
                   fill=c("blue","lightblue"),
                   filename=NULL, 
                   cex=3.5, 
                   col="transparent",
                   cat.cex=0.01,
                   margin=0.01, 
                   main.cex=3, 
                   na='remove');
#grid.draw(vp) 
dev.off()
}

In [45]:
if(bool_plot){
#pdf(file="venn_hpt.pdf", width=8, height=8)
vp<-venn.diagram(list(t=rownames(hpt1),
                      p=as.matrix(diffHPT1[,1])),
                  fill=c("blue", "lightblue"),
                  filename=NULL,
                  cex=3.5, 
                  col="transparent",
                  cat.cex=0.01, 
                  margin=0.01,
                  na='remove');
grid.draw(vp) 
#dev.off()
}

In [46]:
if(bool_plot){
#pdf(file="venn_hippo.pdf", width=8, height=8)
vp<-venn.diagram(list(t=rownames(hippo1),
                      p=as.matrix(diffHippo1[,1])),
                 fill=c("blue","lightblue"),
                 filename=NULL,
                 cex=3.5, 
                 col="transparent",
                 cat.cex=0.01,
                 margin=0.01,
                 na='remove');
grid.draw(vp) 
#dev.off()
}

In [47]:
vsd <- as.data.frame(assay(vst(dds, fitType="local", blind=TRUE)))
vsd1 <- vsd[,c(24:35, 1:23)]

In [48]:
astro = c('Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9', 'Gfap', 'Aldh1l1', 'Gja1', 'Gjb6', 'Agt', 'Atp1b2')
neuro = c('Syp', 'Tubb3','Snap25','Syt1')
microglia = c('Itgam','Aif1')
oligodendrocyte = c('Mog','Mag')
endothelial = c('Slco1c1')
mural = c('Mustn1','Pdgfrb','Des')
tanycyes = c('Crym', 'Ppp1r1b','Rax','Dio2','Slc16a2','Col23a1','Lhx2','Ptm')
npc = c('Nes','Sox2','Prom1')
vlmc = c('Col1a1')

In [49]:
datfin <- data.frame(CTX=apply(vsd1 %>% select(starts_with("ctx_")), 1, mean),
                     HPT=apply(vsd1 %>% select(starts_with("hpt_")), 1, mean),
                     Hippo=apply(vsd1 %>% select(starts_with("hippo_")), 1, mean))

In [50]:
astro = datfin[rownames(datfin)%in%c('Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9', 'Gfap', 'Aldh1l1', 'Gja1', 'Gjb6', 'Agt','Atp1b2'),]
neuro = datfin[rownames(datfin)%in%c('Syp', 'Tubb3', 'Snap25', 'Syt1'),]
mglia = datfin[rownames(datfin)%in%c('Itgam', 'Aif1'),]
oligo = datfin[rownames(datfin)%in%c('Mog', 'Mag'),]
endo = datfin[rownames(datfin)%in%c('Slco1c1'),]
mural = datfin[rownames(datfin)%in%c('Mustn1', 'Pdgfrb', 'Des'),]
tcyes = datfin[rownames(datfin)%in%c('Crym'),]
npc = datfin[rownames(datfin)%in%c('Nes', 'Sox2', 'Prom1'),]
vlmc = datfin[rownames(datfin)%in%c('Col1a1'),]

In [51]:
astro <- astro[order(row.names(astro)),]
neuro <- neuro[order(row.names(neuro)),]
mglia <- mglia[order(row.names(mglia)),]
oligo <- oligo[order(row.names(oligo)),]
mural <- mural[order(row.names(mural)),]
tcyes <- tcyes[order(row.names(tcyes)),]

In [52]:
options(repr.plot.width=5, repr.plot.height=8)

In [53]:
mat<-as.matrix(rbind(astro, neuro, mglia, oligo, endo, mural)) # npc,vlmc,tcyes
    
Omics<-c(rep("transcriptomics", 3))
Acsa2<-rep("Acsa2_pos", 3)
Tissue<-c("cortex", "hypothalamus", "hippocampus")
dcol<-as.data.frame(cbind(Tissue,Acsa2,Omics))
rownames(dcol)<-colnames(mat)

pheatmap(mat, 
         scale="column",
         show_rownames=F, 
         show_colnames=F,
         color=my_palette, 
         cluster_cols=F,
         cluster_rows=F, 
         gaps_col=3,
         gaps_row=c(11,15,17,19,20,23), 
         fontsize=18,
         filename="heatmap_ACSA2.pdf",
         legend=F,
         width=3.3, 
         height=8
) 

# Heatmap clustering and enrichment

### Per tissue heatmap

In [54]:
options(repr.plot.width=5, repr.plot.height=8)

In [55]:
vsd_hpt<-vsd[rownames(vsd)%in%rownames(hpt1),] 
pheatmap(vsd_hpt[,1:11], 
         scale="row",
         show_rownames=FALSE,
         show_colnames=FALSE,
         color=my_palette, 
         cluster_cols=F,
         legend=FALSE,
         fontsize=15,
         filename="heatmap_hpt.pdf",
         width=5,
         height=8,
         breaks=seq(-2, 2, length.out=255)) 

In [56]:
vsd_hip <- vsd[rownames(vsd)%in%rownames(hippo1),] 
pheatmap(vsd_hip[,12:23], 
         scale = "row", 
         show_rownames=FALSE,
         show_colnames=FALSE,
         color=my_palette,
         cluster_cols=F, 
         legend=FALSE,
         fontsize=15, 
         filename="heatmap_hippo.pdf",
         width=5, 
         height=8,
         breaks=seq(-2, 2, length.out=255)
) 

In [57]:
vsd_ctx<-vsd[rownames(vsd)%in%rownames(ctx1),] 
pheatmap(vsd_ctx[,24:35],
             scale="row",
             show_rownames=FALSE,
             show_colnames=FALSE,
             color=my_palette, 
             cluster_cols=F,
             legend=F,
             fontsize=15,
             filename="heatmap_ctx.pdf",
             width=5, 
             height=8,
             breaks=seq(-2, 2, length.out=255)
)

## GO - pathway enrichment

In [58]:
sg.ctx<-bitr(rownames(vsd_ctx), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
go.ctx<-enrichGO(sg.ctx[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

sg.hpt<-bitr(rownames(vsd_hpt), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
go.hpt<-enrichGO(sg.hpt[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

sg.hip<-bitr(rownames(vsd_hip), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
go.hip<-enrichGO(sg.hip[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(vsd_ctx), fromType = "SYMBOL", toType = "ENTREZID", :
“2.63% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(vsd_hpt), fromType = "SYMBOL", toType = "ENTREZID", :
“2.26% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(vsd_hip), fromType = "SYMBOL", toType = "ENTREZID", :
“5.15% of input gene IDs are fail to map...”


In [59]:
go <- c("insulin receptor signaling pathway",
        "response to insulin",
        "TOR signaling",
        "pyruvate metabolic process",
        "intracellular lipid transport",
        "fatty acid metabolic process",
        "fatty acid oxidation",
        "response to reactive oxygen species",
        "response to endoplasmic reticulum stress",
        "regulation of mitotic cell cycle",
        "transforming growth factor beta receptor signaling pathway",
        "I-kappaB kinase/NF-kappaB signaling",
        "regulation of inflammatory response",
        "regulation of cell junction assembly",
        "cell-matrix adhesion",
        "regulation of cytosolic calcium ion concentration",
        "vesicle budding from membrane",
        "axon ensheathment",
        "maintenance of synapse structure",
        "regulation of synapse structure or activity",
        "synaptic transmission, GABAergic",
        "AMPA glutamate receptor clustering",
        "positive regulation of blood vessel endothelial cell migration",
        "sprouting angiogenesis",
        "regulation of angiogenesis",
        "locomotory behavior",
        "feeding behavior"
)
#"cellular carbohydrate metabolic process" ,
#"response to interleukin-1", 
#"cell junction assembly",
#"regulation of neurotransmitter levels", 
#"positive regulation of endothelial cell migration",
#"circadian rhythm",
# ,"regulation of neurotransmitter levels" , 
#"positive regulation of lipid metabolic process",
#"ERK1 and ERK2 cascade",

In [60]:
pdf(file="enrich_path.pdf", width=15, height=15)
    enr_ctx.go=go.ctx@result[go.ctx@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_ctx.go$Tissue='CTX'
    enr_hpt.go=go.hpt@result[go.hpt@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_hpt.go$Tissue='HPT'
    enr_hip.go=go.hip@result[go.hip@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_hip.go$Tissue='Hippo'
    
    all <- rbind(enr_ctx.go, enr_hpt.go, enr_hip.go)
    #write.csv(all, file="GO_transcriptome.csv")
    all <- all[all$Description%in%go,]
    options(repr.plot.width=20, repr.plot.height=15)
    colnames(all) <- c('Description', 'pvalue', 'p.adjust', 'geneID', 'count', 'Tissue')
    ggplot(all,   
           aes(Tissue, y=factor(Description, level=go[length(go):1]), col=pvalue, size=count)) + 
           geom_point() + 
           scale_colour_gradient(low="red", high="blue") +
           scale_size_continuous(range=c(3, 10)) +
           xlab("") + 
           ylab("") + 
           theme(axis.text.x=element_blank()) + theme(legend.position="none") +
           theme(text=element_text(size=40))
dev.off()

pdf 
  2

In [61]:
options(repr.plot.width=7, repr.plot.height=15)

In [62]:
all<-rbind(enr_ctx.go, enr_hpt.go, enr_hip.go)
all<-all[all$Description%in%go,]

In [63]:
sig.gene.hpt.all<-bitr(rownames(hpt), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
sig.gene.ctx.all<-bitr(rownames(ctx), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
sig.gene.hip.all<-bitr(rownames(hippo), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(hpt), fromType = "SYMBOL", toType = "ENTREZID", :
“3.93% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(ctx), fromType = "SYMBOL", toType = "ENTREZID", :
“3.93% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(hippo), fromType = "SYMBOL", toType = "ENTREZID", :
“3.93% of input gene IDs are fail to map...”


In [64]:
pdf(file="enrichdot_ctx.pdf", width=7, height=15)
    df_total.ctx=data.frame()
    for(i in 1:nrow(all)){   
        p<-as.data.frame(ctx[which(rownames(ctx)%in%
                         sig.gene.ctx.all[sig.gene.ctx.all[,2]%in%unlist(strsplit(all$geneID[i],"/")),1]), c(2,5)])
        p$pathway<-all$Description[i]
        p$padjPath<-all$p.adjust[i]
        p$count<-all$Count[i]
        df_total.ctx<-rbind(df_total.ctx, p)    
    }
    df_total.ctx$significance<-ifelse(df_total.ctx$pvalue<0.05, "significant", "not significant")

    ggplot(df_total.ctx, aes(log2FoldChange, y=factor(pathway, level=go[length(go):1]))) + 
           geom_point(aes(colour=significance, size=-log10(pvalue)), alpha=0.5) + 
           scale_size_continuous(range=c(0,8)) +
           scale_color_manual(values=c("gray85", "red")) +
           geom_vline(xintercept=0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text=element_text(size=35)) + #guides(color = guide_legend(override.aes = list(size=5) ) ) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68, 3.82)
dev.off() 

Warning message:
“Removed 12 rows containing missing values (geom_point).”


pdf 
  2

In [65]:
pdf(file="enrichdot_hpt.pdf", width=7, height=15)
    df_total.hpt = data.frame()
    for(i in 1:nrow(all)){
        p<-as.data.frame(hpt[which(rownames(hpt)%in%
                         sig.gene.hpt.all[sig.gene.hpt.all[,2]%in%unlist(strsplit(all$geneID[i],"/")),1]),c(2,5)])
        p$pathway<-all$Description[i]
        p$padjPath<-all$p.adjust[i]
        p$count<-all$Count[i]
        df_total.hpt<-rbind(df_total.hpt, p)    
    } 
    df_total.hpt$col<-ifelse(df_total.hpt$pvalue<0.05, "red", "grey")

    ggplot(df_total.hpt, aes(log2FoldChange, y=factor(pathway, level=go[length(go):1]))) + 
           geom_point(aes(colour=col, size=-log10(pvalue)), alpha=0.5) + 
           scale_size_continuous(range=c(0, 8)) +
           scale_color_manual(values=c("gray85", "red")) + 
           geom_vline(xintercept=0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text=element_text(size=35)) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68,3.82)
dev.off()

Warning message:
“Removed 10 rows containing missing values (geom_point).”


pdf 
  2

In [66]:
pdf(file = "enrichdot_hippo.pdf", width=7, height=15)
    df_total.hip=data.frame()
    for(i in 1:nrow(all)){
        p<-as.data.frame(hippo[which(rownames(hippo)%in%
                         sig.gene.hip.all[sig.gene.hip.all[,2]%in%unlist(strsplit(all$geneID[i],"/")),1]), c(2,5)])
        p$pathway<-all$Description[i]
        p$padjPath<-all$p.adjust[i]
        p$count<-all$Count[i]
        df_total.hip<-rbind(df_total.hip, p)    
    }    
    df_total.hip$col<-ifelse(df_total.hip$pvalue<0.05, "red", "grey")

    ggplot(df_total.hip, aes(log2FoldChange, y=factor(pathway, level=go[length(go):1]))) + 
           geom_point(aes(colour=col, size=-log10(pvalue)), alpha=0.5) + 
           scale_color_manual(values=c("gray85", "red")) + 
           scale_size_continuous(range = c(0, 6)) + 
           geom_vline(xintercept = 0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text = element_text(size=35)) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68, 3.82)
dev.off()

Warning message:
“Removed 15 rows containing missing values (geom_point).”


pdf 
  2

In [67]:
#ggplot(df_total.ctx, aes(log2FoldChange, y = factor(pathway, level = go[length(go):1]))) + 
#       scale_color_gradient(low="gray90", high="red3", limits = c(0,7.1)) +
#       geom_point(aes(color = -log10(pvalue)), size=7, alpha = 0.8) + 
#       geom_vline(xintercept = 0)+
#       theme(axis.title=element_text(size=20,face="bold"),
#             title=element_text(size=20,face="bold"), 
#             text = element_text(size=35)) +
#        ylab("") + theme(legend.position = "none") +
#       xlab("") + theme(axis.text.y=element_blank()) + xlim(-6.5,4.7)

### KEGG

In [ ]:
#kegg <- c("Apoptosis","Calcium signaling pathway","Circadian rhythm","Adherens junction","ECM-receptor interaction",
#          "Endocytosis","AMPK signaling pathway","Fructose and mannose metabolism","Glycolysis / Gluconeogenesis",
#          "Pyruvate metabolism","Hedgehog signaling pathway","GnRH signaling pathway",
#          "Growth hormone synthesis, secretion and action","Thyroid hormone signaling pathway",
#          "TGF-beta signaling pathway",
#          "Adipocytokine signaling pathway","PPAR signaling pathway",
#          "Insulin signaling pathway","mTOR signaling pathway","Wnt signaling pathway","FoxO signaling pathway",
#          "Insulin resistance","PI3K-Akt signaling pathway","VEGF signaling pathway",
#          "HIF-1 signaling pathway","Cholinergic synapse","Dopaminergic synapse","Axon guidance",
#          "GABAergic synapse","Neuroactive ligand-receptor interaction","Oxytocin signaling pathway","Thermogenesis")
#         ,"Fatty acid metabolism","NF-kappa B signaling pathway","Chemokine signaling pathway","TNF signaling pathway",

In [ ]:
#if(bool_plot){  
#    enr_ctx.kegg = kegg.ctx@result[kegg.ctx@result[,5]<0.05,c(2,5,6,8,9)]
#    enr_ctx.kegg$Tissue = 'CTX'
#    enr_hpt.kegg = kegg.hpt@result[kegg.hpt@result[,5]<0.05,c(2,5,6,8,9)]
#    enr_hpt.kegg$Tissue = 'HPT'
#    enr_hip.kegg = kegg.hip@result[kegg.hip@result[,5]<0.05,c(2,5,6,8,9)]
#    enr_hip.kegg$Tissue = 'Hippo'
#
#    all <- rbind(enr_ctx.kegg, enr_hpt.kegg, enr_hip.kegg)
#    #write.csv(all, file="KEGG_transcriptom.csv")
#    all <- all[all$Description%in%kegg,]
#    options(repr.plot.width = 10, repr.plot.height = 15)
#    ggplot(all, 
#           aes(Tissue, Description, col = pvalue, size = Count)) +  # p.adjust  pvalue
#           geom_point()+ 
#           scale_colour_gradient(low = "red", high = "blue") +
#           xlab("Tissue") + 
#           ylab("KEGG enriched pathways") + 
#          theme(text = element_text(size=25),axis.text.x = element_text(angle=45, hjust=1))
#}

In [ ]:
#if(bool_plot){  
#    options(repr.plot.width = 12, repr.plot.height = 15)
#
#    all <- rbind(enr_ctx.kegg, enr_hpt.kegg, enr_hip.kegg)
#    all <- all[all$Description%in%kegg,]
#
#    sig.gene.hpt.all <- bitr(rownames(hpt), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
#    sig.gene.ctx.all <- bitr(rownames(ctx), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
#    sig.gene.hip.all <- bitr(rownames(hippo), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
#}

In [ ]:
#if(bool_plot){   
#    df_total.ctx = data.frame()
#    for(i in 1:nrow(all)){   
#        names <- keggGet(substr(rownames(all)[i],1,8))[[1]]$GENE
#        namesodd <-  names[seq(0,length(names),2)]
#        p <- ctx[ctx[,1]%in%gsub("\\;.*","",namesodd),c(2,5)]
#        
#        p$pathway <- all$Description[i]
#        p$padjPath <- all$p.adjust[i]
#        p$count <- all$Count[i]
#        df_total.ctx <- rbind(df_total.ctx,p)
#    } 
#    ggplot(df_total.ctx, aes(log2FoldChange, pathway)) + 
#           scale_color_gradient(low="grey", high="red") +
#           geom_point(aes(color = -log10(pvalue))) +
#           geom_vline(xintercept = 0)+
#           theme(axis.title=element_text(size=20,face="bold"),
#                 title=element_text(size=20,face="bold"), 
#                 text = element_text(size=25)) +
#           ylab("Pathway Names") +
#           ggtitle("CTX") + 
#           xlab("log2FC")
#}

In [ ]:
#if(bool_plot){       
#    df_total.hpt = data.frame()
#    for(i in 1:nrow(all)){
#        names <- keggGet(substr(rownames(all)[i],1,8))[[1]]$GENE
#        namesodd <-  names[seq(0,length(names),2)]
#        p <- hpt[hpt[,1]%in%gsub("\\;.*","",namesodd),c(2,5)]
#        
#        p$pathway <- all$Description[i]
#        p$padjPath <- all$p.adjust[i]
#        p$count <- all$Count[i]
#        df_total.hpt <- rbind(df_total.hpt,p)
#    }
#    ggplot(df_total.hpt, aes(log2FoldChange, pathway)) +
#           scale_color_gradient(low="grey", high="red") +
#           geom_point(aes(color = -log10(pvalue))) +
#           geom_vline(xintercept = 0)+
#           theme(axis.title=element_text(size=20,face="bold"),
#                 title=element_text(size=20,face="bold"),
#                 text = element_text(size=25)) +
#           ylab("Pathway Names") +
#           ggtitle("HPT") +
#           xlab("log2FC")
#}

In [ ]:
#if(bool_plot){ 
#    df_total.hip = data.frame()
#    for(i in 1:nrow(all)){
#        names <- keggGet(substr(rownames(all)[i],1,8))[[1]]$GENE
#        namesodd <-  names[seq(0,length(names),2)]
#        p <- hippo[hippo[,1]%in%gsub("\\;.*","",namesodd),c(2,5)]
#        
#        p$pathway <- all$Description[i]
#        p$padjPath <- all$p.adjust[i]
#        p$count <- all$Count[i]
#        df_total.hip <- rbind(df_total.hip,p)
#    }
#    ggplot(df_total.hip, aes(log2FoldChange, pathway)) +
#           scale_color_gradient(low="grey", high="red") +
#           geom_point(aes(color = -log10(pvalue))) +
#           geom_vline(xintercept = 0)+
#           theme(axis.title=element_text(size=20,face="bold"),
#                 title=element_text(size=20,face="bold"),
#                 text = element_text(size=25)) +
#           ylab("Pathway Names") +
#           ggtitle("Hippo") +
#           xlab("log2FC")
#}

In [ ]:
#if(bool_plot){
#    gen_heatmap =union(union(hpt[hpt$pvalue<0.05,1], ctx[ctx$pvalue<0.05,1]), hippo[hippo$pvalue<0.05,1])
#    vsd_cl <- vsd[rownames(vsd)%in%gen_heatmap,]
#    hr <- hclust(dist(vsd_cl, method = "maximum"), method="ward.D") 
#    mycl.r <- cutree(hr, k=4)
#    
#    options(repr.plot.width=10, repr.plot.height=10)
#    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) 
#    df <- as.data.frame(colData(dds)[,c("group", "loc")])
#    annotation_row = data.frame(Cluster = factor(mycl.r))
#
#    pheatmap(vsd_cl, cluster_rows=hr, scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=4, 
#             cluster_cols=F, annotation_col=df, annotation_row=annotation_row, fontsize=8,main="All samples",
#             breaks = seq(-4, 4, length.out = 255))
#}

In [ ]:
#if(bool_plot){       
#    df_total.hpt = data.frame()
#    for(i in 1:nrow(all)){
#        p <- as.data.frame(hpt[which(rownames(hpt)%in%
#                                  sig.gene.hpt.all[sig.gene.hpt.all[,2]%in%unlist(strsplit(all$geneID[i],"/")),1]),c(2,5)])
#        p$pathway <- all$Description[i]
#        p$padjPath <- all$p.adjust[i]
#        p$count <- all$Count[i]
#        df_total.hpt <- rbind(df_total.hpt,p)
#    }
#    df_total.hpt$log2FoldChange  <-ifelse(df_total.hpt$log2FoldChange>8,8,df_total.ctx$log2FoldChange )
#    df_total.hpt$log2FoldChange  <-ifelse(df_total.hpt$log2FoldChange< -8,-8,df_total.ctx$log2FoldChange )
#    df_total.hpt$pvalue  <-ifelse(df_total.hpt$pvalue<1e-05,1e-05,df_total.ctx$pvalue )
#
#    ggplot(df_total.hpt, aes(log2FoldChange, pathway)) +
#           scale_color_gradient(low="grey", high="red") +
#           geom_point(aes(color = -log10(pvalue))) +
#           geom_vline(xintercept = 0)+
#           theme(axis.title=element_text(size=20,face="bold"),
#                 title=element_text(size=20,face="bold"),
#                 text = element_text(size=25)) +
#           ylab("Pathway Names") +
#           ggtitle("Significant Genes") +
#           xlab("log2FC")
#}